<a href="https://colab.research.google.com/github/farrelrassya/Machine-Learning-with-Imbalanced-Dataset-Udemy/blob/main/Section%203%20Metrics%20Evaluation%20/11_Multiclass_Matriks_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$
\text{Precision} = \frac{\text{tp}}{\text{tp} + \text{fp}}
$$

$$
\text{Recall} = \frac{\text{tp}}{\text{tp} + \text{fn}}
$$

$$
\text{F1} = \frac{2 \times (\text{Precision} \times \text{Recall})}{\text{Precision} + \text{Recall}}
$$

$$
\text{Support} = \text{Number of cases on each class}
$$


By default, sklearn determines the class as the observation with the highest probability value. In this case, it does not depend on a specific threshold value.

In this notebook, we will obtain the values of the metrics:

1. per class
2. macro averaged
3. micro averaged

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    balanced_accuracy_score,
)

In [2]:
# load data
data = load_wine()

data = pd.concat([
    pd.DataFrame(data.data, columns=data.feature_names),
    pd.DataFrame(data.target, columns=['target']),
    ], axis=1)

data.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [3]:
# target distribution:
# multiclass and (fairly) balanced

data.target.value_counts(normalize=True)

1    0.398876
0    0.331461
2    0.269663
Name: target, dtype: float64

In [4]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),  # drop the target
    data['target'],  # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((124, 13), (54, 13))

Train ML models
Random Forests
Produce probability vectors for each class.

In [5]:
# set up the model
rf = RandomForestClassifier(n_estimators=10, random_state=39, max_depth=1, n_jobs=4)

# train the model
rf.fit(X_train, y_train)

# produce the predictions (as probabilities)
y_train_rf = rf.predict_proba(X_train)
y_test_rf = rf.predict_proba(X_test)

# note that the predictions are an array of 3 columns

# first column: the probability of an observation of being of class 0
# second column: the probability of an observation of being of class 1
# third column: the probability of an observation of being of class 2

y_test_rf[0:10, :]

array([[0.59291486, 0.35444264, 0.0526425 ],
       [0.12139867, 0.33577091, 0.54283043],
       [0.30482504, 0.55905479, 0.13612017],
       [0.52711941, 0.38876082, 0.08411977],
       [0.27876443, 0.50875176, 0.21248381],
       [0.34573413, 0.49743863, 0.15682724],
       [0.51144556, 0.3911751 , 0.09737934],
       [0.034061  , 0.34869659, 0.6172424 ],
       [0.22335574, 0.59578725, 0.18085702],
       [0.22335574, 0.59578725, 0.18085702]])

In [6]:
# The final prediction is that of the biggest probabiity

rf.predict(X_test)[0:10]


array([0, 2, 1, 0, 1, 1, 0, 2, 1, 1])

Logistic Regression
The Logistic regression also support multiclass targets.

In [7]:
# set up the model
logit = LogisticRegression(
    random_state=0, multi_class='multinomial', max_iter=100,
)

# train
logit.fit(X_train, y_train)

# obtain the probabilities
y_train_logit = logit.predict_proba(X_train)
y_test_logit = logit.predict_proba(X_test)

# note that the predictions are an array of 3 columns

# first column: the probability of an observation of being of class 0
# second column: the probability of an observation of being of class 1
# third column: the probability of an observation of being of class 2

y_test_logit[0:10, :]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[9.91174785e-01, 1.25204578e-03, 7.57316964e-03],
       [2.39961135e-06, 9.26931569e-08, 9.99997508e-01],
       [6.72160405e-03, 9.91784001e-01, 1.49439529e-03],
       [9.85727996e-01, 2.01748856e-03, 1.22545157e-02],
       [4.70970476e-03, 9.93599098e-01, 1.69119773e-03],
       [7.82238109e-04, 9.92151343e-01, 7.06641861e-03],
       [9.99368867e-01, 1.30515090e-04, 5.00618297e-04],
       [7.52856238e-04, 5.64774337e-05, 9.99190666e-01],
       [1.56450691e-02, 9.81843076e-01, 2.51185498e-03],
       [4.08699212e-03, 9.94374282e-01, 1.53872540e-03]])

In [8]:
# The final prediction is that of the biggest probabiity

logit.predict(X_test)[0:10]

array([0, 2, 1, 0, 1, 1, 0, 2, 1, 1])

In [9]:
# For the rest of the notebook, we work with the class predictions

y_rf_pred = rf.predict(X_test)

y_logit_pred = logit.predict(X_test)

Metrics For each class

In [10]:
p, r, f, s = precision_recall_fscore_support(
    y_test,
    y_rf_pred,
    labels=[0,1,2], # the labels for which we want the metrics determined
    average=None, # when None, returns a metric per label
)

print('Precision: ', p)
print('Recall: ', r)
print('f score: ', f)
print('Support: ', s)
print()

Precision:  [1.         0.86956522 0.85714286]
Recall:  [0.89473684 0.90909091 0.92307692]
f score:  [0.94444444 0.88888889 0.88888889]
Support:  [19 22 13]



Macro
Take the average of the individual metrics

In [11]:
p, r, f, s = precision_recall_fscore_support(
    y_test,
    y_rf_pred,
    labels=[0,1,2], # the labels for which we want the metrics determined
    average='macro', # take the average of the metrics
)

print('Precision: ', p)
print('Recall: ', r)
print('f score: ', f)
print('Support: ', s)
print()

Precision:  0.9089026915113871
Recall:  0.9089682247576985
f score:  0.9074074074074074
Support:  None



In [12]:
# For precision this is the same as:

(1. + 0.86956522 + 0.85714286) / 3

0.9089026933333333

Weighed average
Takes the average of each metric weighted by the support.

In [13]:
p, r, f, s = precision_recall_fscore_support(
    y_test,
    y_rf_pred,
    labels=[0,1,2], # the labels for which we want the metrics determined
    average='weighted', # take the average of the metrics
)

print('Precision: ', p)
print('Recall: ', r)
print('f score: ', f)
print('Support: ', s)
print()

Precision:  0.9124683689901082
Recall:  0.9074074074074074
f score:  0.9084362139917695
Support:  None



In [14]:
# For precision this is the same as:

(19 * 1. + 22 * 0.86956522 + 13 * 0.85714286) / (19+22+13)

0.9124683707407407

Micro
Collective average of TP, FP anf FN.

In [15]:
# we determine the metrics for each one of the classes
# just like we did in the intro video

p, r, f, s = precision_recall_fscore_support(
    y_test, y_rf_pred, labels=[0,1,2], average='micro',
)

print('Precision: ', p)
print('Recall: ', r)
print('f score: ', f)
print('Support: ', s)
print()

Precision:  0.9074074074074074
Recall:  0.9074074074074074
f score:  0.9074074074074074
Support:  None



Model comparison
Recall, Precision and f score

In [16]:
# random forests

p, r, f, s = precision_recall_fscore_support(
    y_test,
    y_rf_pred,
    labels=[0,1,2], # the labels for which we want the metrics determined
    average='macro', # take the average of the metrics
)

print('Precision: ', p)
print('Recall: ', r)
print('f score: ', f)
print('Support: ', s)
print()

Precision:  0.9089026915113871
Recall:  0.9089682247576985
f score:  0.9074074074074074
Support:  None



In [17]:
# logistic regression

p, r, f, s = precision_recall_fscore_support(
    y_test,
    y_logit_pred,
    labels=[0,1,2], # the labels for which we want the metrics determined
    average='macro', # take the average of the metrics
)

print('Precision: ', p)
print('Recall: ', r)
print('f score: ', f)
print('Support: ', s)
print()

Precision:  0.9472049689440993
Recall:  0.9497607655502392
f score:  0.9469135802469136
Support:  None



The Logistic regression seems to be working a bit better on average.



In [18]:
print('Accuracy Random Forest test:', accuracy_score(y_test, rf.predict(X_test)))
print('Accuracy Logistic Regression test:', accuracy_score(y_test, logit.predict(X_test)))

Accuracy Random Forest test: 0.9074074074074074
Accuracy Logistic Regression test: 0.9444444444444444


In [19]:
print('Balanced accuracy, Random Forest test:', balanced_accuracy_score(y_test, rf.predict(X_test)))
print('Balanced accuracy, Regression test:',  balanced_accuracy_score(y_test, logit.predict(X_test)))

Balanced accuracy, Random Forest test: 0.9089682247576985
Balanced accuracy, Regression test: 0.9497607655502392
